# Trading Algorítmico con Transformers y Backtrader

Este notebook sigue el tutorial `transformer_tutorial.md`. Aquí implementaremos el flujo de trabajo completo: desde la preparación de los datos y el entrenamiento del modelo Transformer hasta la creación y ejecución de una estrategia de trading en `backtrader`.

## 1. Cargar y Preparar los Datos

In [ ]:
import matplotlib
matplotlib.use('Agg') # Configura Matplotlib para no usar una GUI interactiva. Esencial para scripts.
import pandas as pd # Importa pandas para la manipulación de datos, una herramienta clave en data science.
import pandas_ta as ta # Extiende pandas con funciones de análisis técnico.
import numpy as np # Importa numpy para operaciones numéricas eficientes.
from sklearn.preprocessing import MinMaxScaler # Para normalizar los datos, un paso crucial en el preprocesamiento.
import tensorflow as tf # El framework de deep learning que usaremos para construir el Transformer.
from tensorflow.keras.models import Model, load_model # Funciones para definir y cargar modelos de Keras.
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D # Capas para el modelo.
import matplotlib.pyplot as plt # Para la visualización de datos y resultados.
import backtrader as bt # El framework de backtesting para probar nuestra estrategia de trading.

# Carga de datos desde un CSV. `index_col` y `parse_dates` son para manejar correctamente las fechas.
df = pd.read_csv('../yahoo/AAPL.csv', index_col='datetime', parse_dates=True)

# Cálculo de indicadores técnicos. `append=True` los añade como nuevas columnas al DataFrame.
df.ta.rsi(length=14, append=True) # Relative Strength Index (RSI)
df.ta.macd(fast=12, slow=26, signal=9, append=True) # Moving Average Convergence Divergence (MACD)
df.ta.bbands(length=20, append=True) # Bollinger Bands
df.ta.atr(length=14, append=True) # Average True Range (ATR)
df.ta.stoch(length=14, append=True) # Stochastic Oscillator

# Eliminamos las filas con valores NaN, que aparecen por el cálculo de los indicadores con ventanas de tiempo.
df.dropna(inplace=True)

# Definimos las características (features) que alimentarán nuestro modelo.
features = [
    'close', 'RSI_14', 'MACD_12_26_9', 'BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0', 
    'ATRr_14', 'STOCHk_14_3_3', 'STOCHd_14_3_3'
]

# pandas_ta a veces crea nombres de columna ligeramente diferentes. Este bucle los corrige.
for col in df.columns:
    if 'BBL_20_2.0' in col: features[3] = col
    if 'BBM_20_2.0' in col: features[4] = col
    if 'BBU_20_2.0' in col: features[5] = col

# Creamos un nuevo DataFrame `data` solo con las features que nos interesan.
data = df[features]
n_features = len(features) # Guardamos el número de características.

# Normalizamos los datos para que estén en un rango de 0 a 1. Esto mejora el rendimiento del modelo.
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Creamos un segundo scaler solo para el precio de cierre. Lo usaremos para des-normalizar las predicciones.
scaler_pred = MinMaxScaler(feature_range=(0, 1))
scaler_pred.fit_transform(data[['close']])

# Función para crear secuencias de datos. Los modelos de series temporales necesitan datos en este formato.
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length]) # `X` es una secuencia de `seq_length` pasos.
        y.append(data[i + seq_length, 0]) # `y` es el precio de cierre del siguiente paso.
    return np.array(X), np.array(y)

# Definimos la longitud de la secuencia. Usaremos 60 días de datos para predecir el siguiente.
SEQ_LENGTH = 60
X, y = create_sequences(scaled_data, SEQ_LENGTH)

# Dividimos los datos en un conjunto de entrenamiento (80%) y uno de prueba (20%).
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

## 2. Construir y Entrenar el Modelo Transformer

In [ ]:
# Bloque codificador del Transformer. Contiene atención multi-cabeza y una red feed-forward.
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalización y Atención Multi-cabeza
    x = LayerNormalization(epsilon=1e-6)(inputs)
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = Dropout(dropout)(x)
    res = x + inputs # Conexión residual

    # Red Feed-Forward
    x = LayerNormalization(epsilon=1e-6)(res)
    x = Dense(ff_dim, activation="relu")(x)
    x = Dropout(dropout)(x)
    x = Dense(inputs.shape[-1])(x)
    return x + res # Segunda conexión residual

# Función para construir el modelo Transformer completo.
def build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = Input(shape=input_shape)
    x = inputs
    # Apilamos varios bloques de codificador.
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    # Pooling para reducir la dimensionalidad y una red MLP para la salida final.
    x = GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = Dense(dim, activation="relu")(x)
        x = Dropout(mlp_dropout)(x)
    outputs = Dense(1)(x) # La salida es un único valor: el precio predicho.
    return Model(inputs, outputs)

# Construimos el modelo con los hiperparámetros definidos.
model = build_transformer_model(
    (SEQ_LENGTH, n_features), head_size=128, num_heads=4, ff_dim=4,
    num_transformer_blocks=4, mlp_units=[64], dropout=0.1, mlp_dropout=0.1
)

# Compilamos el modelo. Usamos Adam como optimizador y el error cuadrático medio como función de pérdida.
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary() # Mostramos un resumen de la arquitectura del modelo.

# Entrenamos el modelo con los datos de entrenamiento.
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

## 3. Guardar el Modelo

In [ ]:
# Guardamos el modelo entrenado para poder usarlo más tarde sin necesidad de re-entrenar.
model.save('transformer_model.keras')

## 4. Evaluar el Modelo

In [ ]:
# Realizamos predicciones sobre el conjunto de prueba.
predictions = model.predict(X_test)
# Des-normalizamos las predicciones para que estén en la escala original de precios.
predictions = scaler_pred.inverse_transform(predictions)
y_test_inv = scaler_pred.inverse_transform(y_test.reshape(-1, 1))

# Visualizamos los resultados para comparar el precio real con el predicho.
plt.figure(figsize=(14, 5))
plot_index = data.index[train_size + SEQ_LENGTH:] # Obtenemos los índices de fecha correctos para el plot.
plt.plot(plot_index, y_test_inv, color='blue', label='Precio Real')
plt.plot(plot_index, predictions, color='red', label='Precio Predicho')
plt.title('Predicción de Precios con Transformer')
plt.xlabel('Fecha')
plt.ylabel('Precio')
plt.legend()
# Guardamos el gráfico como una imagen.
plt.savefig('prediction_plot.png')

## 5. Estrategia en backtrader

In [ ]:
# Definimos la estrategia de trading para backtrader.
class TransformerStrategy(bt.Strategy):
    def __init__(self):
        # Cargamos el modelo y los scalers al iniciar la estrategia.
        self.model = load_model('transformer_model.keras')
        self.scaler = scaler
        self.scaler_pred = scaler_pred
        self.seq_length = SEQ_LENGTH

        # Creamos referencias a los datos y a los indicadores que usaremos.
        self.data_close = self.datas[0].close
        self.rsi = bt.indicators.RSI(self.datas[0], period=14)
        self.macd = bt.indicators.MACD(self.datas[0], period_me1=12, period_me2=26, period_signal=9)
        self.bbands = bt.indicators.BollingerBands(self.datas[0], period=20)
        self.atr = bt.indicators.AverageTrueRange(self.datas[0], period=14)
        self.stoch = bt.indicators.Stochastic(self.datas[0], period=14)

    # El método `next` se ejecuta en cada barra (cada día en este caso).
    def next(self):
        # Esperamos a tener suficientes datos para hacer una predicción.
        if len(self) < self.seq_length + 26: # 26 es un margen por el cálculo de indicadores como el MACD.
            return

        # Recopilamos los últimos `seq_length` valores de nuestros indicadores.
        close_vals = self.data_close.get(size=self.seq_length)
        rsi_vals = self.rsi.get(size=self.seq_length)
        macd_vals = self.macd.macd.get(size=self.seq_length)
        bbl_vals = self.bbands.lines.bot.get(size=self.seq_length)
        bbm_vals = self.bbands.lines.mid.get(size=self.seq_length)
        bbu_vals = self.bbands.lines.top.get(size=self.seq_length)
        atr_vals = self.atr.get(size=self.seq_length)
        stochk_vals = self.stoch.lines.percK.get(size=self.seq_length)
        stochd_vals = self.stoch.lines.percD.get(size=self.seq_length)

        # Creamos el array de entrada para el modelo.
        input_array = np.array([
            close_vals, rsi_vals, macd_vals, bbl_vals, bbm_vals, bbu_vals, 
            atr_vals, stochk_vals, stochd_vals
        ]).T

        # Normalizamos los datos de entrada y hacemos la predicción.
        scaled_input = self.scaler.transform(input_array)
        X_pred = np.array([scaled_input])
        prediction_scaled = self.model.predict(X_pred)
        prediction = self.scaler_pred.inverse_transform(prediction_scaled)[0][0]

        # Lógica de trading: si la predicción es mayor que el precio actual, compramos. Si es menor, vendemos.
        if prediction > self.data_close[0]:
            if not self.position: # Si no tenemos una posición abierta.
                self.buy()
        elif prediction < self.data_close[0]:
            if self.position: # Si tenemos una posición abierta.
                self.sell()

# --- Configuración y ejecución del backtest con Cerebro ---
cerebro = bt.Cerebro() # Creamos una instancia de Cerebro, el motor de backtrader.
data_feed = bt.feeds.PandasData(dataname=df) # Creamos un feed de datos a partir de nuestro DataFrame.
cerebro.adddata(data_feed) # Añadimos los datos a Cerebro.
cerebro.addstrategy(TransformerStrategy) # Añadimos nuestra estrategia.
cerebro.broker.setcash(100000.0) # Configuramos el capital inicial.
cerebro.broker.setcommission(commission=0.001) # Configuramos la comisión por operación.

print(f'Capital Inicial: {cerebro.broker.getvalue()}')
cerebro.run() # Ejecutamos el backtest.
print(f'Capital Final: {cerebro.broker.getvalue()}')

# Guardamos el gráfico del backtest. Se usa un truco para que no intente mostrarlo en pantalla.
_original_show = plt.show
plt.show = lambda: None
fig = cerebro.plot(style='candlestick')[0][0]
fig.savefig('backtest_plot.png')
plt.show = _original_show # Restauramos la función original.